### ДЗ 1 Анна Запорощенко

1. Ссылки и тексты в файле corpus.xlsx. Все тексты представляют собой новостные статьи из www.dw.com Ключевые слова представлены в виде слов и словосочетаний, чаще в начальной форме (иногда допускается множествыенное число). Представляют собой теги-гиперссылки, по которым можно найти новости с пхожей тематикой. 
2. Уже размеченные ключевые слова, выделенные мной, их пересечение и конечный список с комментарием даны в файле corpus.xlsx.

In [128]:
import pandas as pd
import numpy as np
import RAKE
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
import stanza

In [129]:
stop = stopwords.words('russian')
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [130]:
corpus = pd.read_excel(r"corpus.xlsx")

In [131]:
corpus

,Текст,DW ключевые слова,Выделенные ключевые слова,Пересечение,Окончательный список ключевых слов,Комментарий,Ссылка
0,Кто следующий? Бездомность без прикрас\nБездом...,"бездомные, безработные, выставка ""Кто следующи...","бездомность, Мюнхен, бездомные, безработные, в...","бездомные, безработные, Мюнхен, архитектура, в...",бездомность; бездомные; безработные; Мюнхен; а...,Название выставки лучше исключить из-за наличи...,https://www.dw.com/ru/kto-sledujuchij-bezdomno...
1,Когда чрезмерная диагностика и терапия могут н...,"диагностика, чрезмерная диагностика, медицина,...","медицина, чрезмерная диагностика и терапия, оп...","медицина, оперативное вмешательство",медицина; чрезмерная диагностика; терапия; опе...,В целом теги получились практически идентичные...,https://www.dw.com/ru/kogda-chrezmernaja-diagn...
2,Зачем немецкому стартапу хлебные крошки\nКарто...,"немецкая кухня, картофельные кнедли, кулинарны...","кнедли, клопсы, кулинарный стартап, хлеб, реце...","кулинарный стартап, блюда из остатков",кулинарный стартап; блюда из остатков; хлеб; к...,Изначально проставлено очень много общих тегов...,https://www.dw.com/ru/zachem-nemeckomu-startap...
3,Зачем немецким университетам становиться более...,"экологичность, учеба в Германии, образование в...","экологичность, Германия, университет, отказать...",экологичность,экологичность; Германия; университет; климатич...,"""Защита окружающей среды"" (не встречается в т...",https://www.dw.com/ru/zachem-nemeckim-universi...


3. 
#### RAKE

In [102]:
# Инициализируем анализатор списком стоп-слов
rake = RAKE.Rake(stop)

In [103]:
corpus['RAKE'] = np.nan
for num, row in corpus.iterrows():
    r = rake.run(normalize_text(row['Текст']), maxWords=3, minFrequency=2)
    r = [i[0] for i in r][:11]
    r = '; '.join(r)
    corpus.loc[num, 'RAKE'] = r

3. #### TextRank

In [104]:
corpus['TextRank'] = np.nan
for num, row in corpus.iterrows():
    rt = keywords.keywords(normalize_text(row['Текст']), language='russian', additional_stopwords=stop, scores=True)
    rt = [i[0] for i in rt][:11]
    rt = '; '.join(rt)
    corpus.loc[num, 'TextRank'] = rt

3. #### Tf*Idf

In [105]:
vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=stop)
texts = corpus['Текст'].tolist()
texts = [normalize_text(text) for text in texts]
texts = vectorizer.fit_transform(texts)
words = np.array(vectorizer.get_feature_names())
args = np.argsort(texts.toarray())

In [106]:
corpus['TfIdf'] = np.nan
for num, row in corpus.iterrows():
    tfidf = words[args[num]][::-1][:10]
    tfidf = '; '.join(tfidf)
    corpus.loc[num, 'TfIdf'] = tfidf

In [107]:
corpus

,Текст,DW ключевые слова,Выделенные ключевые слова,Пересечение,Окончательный список ключевых слов,Комментарий,Ссылка,RAKE,TextRank,TfIdf
0,Кто следующий? Бездомность без прикрас\nБездом...,"бездомные, безработные, выставка ""Кто следующи...","бездомность, Мюнхен, бездомные, безработные, в...","бездомные, безработные, Мюнхен, архитектура, в...",бездомность; бездомные; безработные; Мюнхен; а...,Название выставки лучше исключить из-за наличи...,https://www.dw.com/ru/kto-sledujuchij-bezdomno...,проблема бездомность; проблема; город; оказать...,бездомность; бездомный; жильё; социальный; пом...,бездомность; проблема; человек; выставка; соци...
1,Когда чрезмерная диагностика и терапия могут н...,"диагностика, чрезмерная диагностика, медицина,...","медицина, чрезмерная диагностика и терапия, оп...","медицина, оперативное вмешательство",медицина; чрезмерная диагностика; терапия; опе...,В целом теги получились практически идентичные...,https://www.dw.com/ru/kogda-chrezmernaja-diagn...,рак простата; чрезмерный диагностика; врач; те...,врач; мочить; который; терапия мочь; здоровье;...,врач; диагностика; лечение; пациент; терапия; ...
2,Зачем немецкому стартапу хлебные крошки\nКарто...,"немецкая кухня, картофельные кнедли, кулинарны...","кнедли, клопсы, кулинарный стартап, хлеб, реце...","кулинарный стартап, блюда из остатков",кулинарный стартап; блюда из остатков; хлеб; к...,Изначально проставлено очень много общих тегов...,https://www.dw.com/ru/zachem-nemeckomu-startap...,рассказывать жанин трапп; сандра ляйтнер; кнед...,кнедлить; кнедлей; кнедль; хлебо; чёрствый хле...,кнедлить; хлеб; хлебный; кнедлей; картофельный...
3,Зачем немецким университетам становиться более...,"экологичность, учеба в Германии, образование в...","экологичность, Германия, университет, отказать...",экологичность,экологичность; Германия; университет; климатич...,"""Защита окружающей среды"" (не встречается в т...",https://www.dw.com/ru/zachem-nemeckim-universi...,университетский столовый; martin herrmann; это...,это; немецкий университет; экологичный; стать;...,вуз; университет; это; командировка; берлин; с...


4. Выберем шаблоны из списка эталонных КС.

In [108]:
corpus['Окончательный список ключевых слов'].tolist()

['бездомность; бездомные; безработные; Мюнхен; архитектура; социальное неравенство; проблема бездомных',
 'медицина; чрезмерная диагностика; терапия; оперативное вмешательство; операция; врачебная ошибка',
 'кулинарный стартап; блюда из остатков; хлеб; кнедли; рецепты; немецкая кухня; клопсы',
 'экологичность; Германия; университет; климатическая нейтральность; отказаться от мяса; зеленое электричество; онлайн']

* NOUN
* ADJ — или субстантивированные прилагательные. Парсер в изоляции может посчитать их обычными прилагательными.
* ADJ + NOUN
* NOUN + NOUN
* NOUN + PREP + NOUN
* VERB + PREP + NOUN

С помощью всех шаблонов будем фильтровать полученные с помощью алгоритмов КС. 

In [109]:
stanza.download("ru")

2021-11-07 21:56:36 INFO: Downloading default packages for language: ru (Russian)...


2021-11-07 21:56:37 INFO: File exists: C:\Users\VADIK\stanza_resources\ru\default.zip.
2021-11-07 21:56:41 INFO: Finished downloading models and saved to C:\Users\VADIK\stanza_resources.


In [110]:
nlp = stanza.Pipeline("ru")
methods = ['RAKE', 'TextRank', 'TfIdf']
for method in methods:
    corpus[method + '_filtered'] = np.nan
    for num, row in corpus.iterrows():
        keywords = row[method].split('; ')
        final_keywords = []
        for keyword in keywords:
            parsed_keyword = nlp(keyword)
            tokens = parsed_keyword.to_dict()[0]
            if ((len(tokens) == 1) and 
                (tokens[0]['upos'] in ['NOUN', 'ADJ'])):
                final_keywords.append(parsed_keyword.text)
            elif ((len(tokens) == 2) and 
                  (tokens[0]['upos'] in ['NOUN', 'ADJ']) and
                  (tokens[1]['upos'] == 'NOUN')):
                final_keywords.append(parsed_keyword.text)
            elif ((len(tokens) == 3) and
                  (tokens[0]['upos'] in ['NOUN', 'VERB']) and
                  (tokens[1]['upos'] == 'PREP') and
                  (tokens[2]['upos'] == 'NOUN')):
                final_keywords.append(parsed_keyword.text)
        final_keywords = '; '.join(final_keywords)
        corpus.loc[num, method + '_filtered'] = final_keywords

2021-11-07 21:56:41 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2021-11-07 21:56:41 INFO: Use device: cpu
2021-11-07 21:56:41 INFO: Loading: tokenize
2021-11-07 21:56:41 INFO: Loading: pos
2021-11-07 21:56:41 INFO: Loading: lemma
2021-11-07 21:56:41 INFO: Loading: depparse
2021-11-07 21:56:41 INFO: Loading: ner
2021-11-07 21:56:42 INFO: Done loading processors!


5. Лемматизируем эталонный список ключевых слов.

In [111]:
for num, row in corpus.iterrows():
    text = row['Окончательный список ключевых слов'].split('; ')
    text = [normalize_text(word) for word in text]
    text = '; '.join(text)
    corpus.loc[num, 'Окончательный список ключевых слов'] = text

In [113]:
rake = set('; '.join(corpus['RAKE'].tolist()).split('; '))
rake_filtered = set('; '.join(corpus['RAKE_filtered'].tolist()).split('; '))
textrank = set('; '.join(corpus['TextRank'].tolist()).split('; '))
textrank_filtered = set('; '.join(corpus['TextRank_filtered'].tolist()).split('; '))
tfidf = set('; '.join(corpus['TfIdf'].tolist()).split('; '))
tfidf_filtered = set('; '.join(corpus['TfIdf_filtered'].tolist()).split('; '))
gold_keywords = set('; '.join(corpus['Окончательный список ключевых слов'].tolist()).split('; '))

RAKE vs RAKE_filtered

In [121]:
TP = len(gold_keywords.intersection(rake))
TP_FP = len(rake)
TP_FN = len(gold_keywords)
precision = TP / TP_FP
recall = TP / TP_FN
f_score = 2 * precision * recall / (precision + recall)

TP_f = len(gold_keywords.intersection(rake_filtered))
TP_FP_f = len(rake_filtered)
TP_FN_f = len(gold_keywords)
precision_f = TP_f / TP_FP_f
recall_f = TP_f / TP_FN_f
f_score_f = 2 * precision_f * recall_f / (precision_f + recall_f)
print('score \t\t RAKE \t\t\t RAKE_filtered')
print('precision \t {} \t {}'.format(precision, precision_f))
print('recall \t\t {} \t {}'.format(recall, recall_f))
print('f_score \t {} \t {}'.format(f_score, f_score_f))

score 		 RAKE 			 RAKE_filtered
precision 	 0.13953488372093023 	 0.16129032258064516
recall 		 0.2222222222222222 	 0.18518518518518517
f_score 	 0.17142857142857143 	 0.17241379310344826


TextRank vs TextRank_filtered

In [123]:
TP = len(gold_keywords.intersection(textrank))
TP_FP = len(textrank)
TP_FN = len(gold_keywords)
precision = TP / TP_FP
recall = TP / TP_FN
f_score = 2 * precision * recall / (precision + recall)

TP_f = len(gold_keywords.intersection(textrank_filtered))
TP_FP_f = len(textrank_filtered)
TP_FN_f = len(gold_keywords)
precision_f = TP_f / TP_FP_f
recall_f = TP_f / TP_FN_f
f_score_f = 2 * precision_f * recall_f / (precision_f + recall_f)
print('score \t\t TextRank \t\t TextRank_filtered')
print('precision \t {} \t {}'.format(precision, precision_f))
print('recall \t\t {} \t {}'.format(recall, recall_f))
print('f_score \t {} \t {}'.format(f_score, f_score_f))

score 		 TextRank 		 TextRank_filtered
precision 	 0.11904761904761904 	 0.14285714285714285
recall 		 0.18518518518518517 	 0.14814814814814814
f_score 	 0.14492753623188406 	 0.14545454545454545


TfIdf vs TfIdf_filtered

In [125]:
TP = len(gold_keywords.intersection(tfidf))
TP_FP = len(tfidf)
TP_FN = len(gold_keywords)
precision = TP / TP_FP
recall = TP / TP_FN
f_score = 2 * precision * recall / (precision + recall)

TP_f = len(gold_keywords.intersection(tfidf_filtered))
TP_FP_f = len(tfidf_filtered)
TP_FN_f = len(gold_keywords)
precision_f = TP_f / TP_FP_f
recall_f = TP_f / TP_FN_f
f_score_f = 2 * precision_f * recall_f / (precision_f + recall_f)
print('score \t\t TfIdf \t\t\t TfIdf_filtered')
print('precision \t {} \t\t\t {}'.format(precision, precision_f))
print('recall \t\t {} \t {}'.format(recall, recall_f))
print('f_score \t {} \t {}'.format(f_score, f_score_f))

score 		 TfIdf 			 TfIdf_filtered
precision 	 0.2 			 0.19444444444444445
recall 		 0.2962962962962963 	 0.25925925925925924
f_score 	 0.23880597014925375 	 0.2222222222222222


6. Анализ ошибок
    * неправильная лемматизация и, соответственно, изменение части речи: кнедли — кнедлить. Такие проблемы немного ухудшают работу отфильтрованных версий, так как по сути правильно выделенные слова будут отфильтровываться шаблоном (так как шаблона VERB не было)
    * выделение слов, которые участвуют в коллокациях, как-то: "проблема бездомных", "бездомный", "проблема". В этом случае слово "проблема" будет считаться ошибочным. Возможно, стоит убирать слова, которые уже входят в коллокации, чтобы не было повторов.
    * выделение слов, которые отображают суть, но являются слишком частными, как-то: zoom. В тексте идет речь о переходе в онлайн-формат и упоминается это программа, но использовать слово zoom как ключевое кажется неправильным, потому что название программы — слишком детальная информация. Можно попытаться использовать иерархические сети, связывающие понятия, как WordNet, и из выделенных ключевых слов пытаться выбрать более общие концепты.
    * плохо выделяются сложные фразы, фразы с предлогами из-за низкой частотности. Возможно, значимость таких фраз стоит попробовать повысить с помощью учета частотности самостоятельных слов, которые в нее входят.
    
Отдельно хочется сказать о том, что ключевые слова в нынешнем виде пригодны для оценки, но не для использования, так как внутри фраз слова стоят в начальной форме. Необходимо либо с помощью шаблонов ставить выделенные КС в нужную форму, либо уже после вычленять из текста формы, в которых они стояли, и после использовать наиболее пригодную (например, ту, где главное слово  именной группы стоит в именительном падеже).